# Installing dependencies


In [31]:
from transformers  import AutoTokenizer,AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re


# Loading Model


In [32]:
tokernizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model= AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")


# Encode and Calculate Sentiment


In [33]:
tokens = tokernizer.encode('It was great,could be better',return_tensors='pt')
tokens

tensor([[  101, 10197, 10140, 11838,   117, 12296, 10346, 16197,   102]])

In [34]:
result = model(tokens)

In [35]:
int(torch.argmax(result.logits))+1

4

# Collect Reviews

In [51]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text,'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p',{'class':regex})
reviews = [result.text for result in results]

# Load Revies into DataFrame and Score

In [52]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.array(reviews),columns=['review'])

In [53]:
df.head()

,review
0,Hotter than I imagine myself to be and firing ...
1,The food and service here was really good. It...
2,Visiting from Texas and decided to give this r...
3,"Great atmosphere, attentive service, solid mar..."
4,Don't come here expecting legit Mexican food b...


In [54]:
def sentiment_score(review):
    tokens = tokernizer.encode(review,return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1


In [55]:
df['score'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [56]:
df

,review,score
0,Hotter than I imagine myself to be and firing ...,5
1,The food and service here was really good. It...,5
2,Visiting from Texas and decided to give this r...,5
3,"Great atmosphere, attentive service, solid mar...",3
4,Don't come here expecting legit Mexican food b...,3
5,Out of all the restaurants that I tried in Syd...,5
6,We came here on a Thursday night @ 5pm and by ...,4
7,The food is fresh and tasty. The scallop cevi...,4
8,Have been here twice and have absolutely loved...,5
9,I was pleasantly surprised at what a great job...,5
